In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from numpy.core.fromnumeric import shape
import keras
from keras.models import Sequential
from keras.layers import Conv1D
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import SimpleRNN
from keras.layers import Flatten
from keras.layers import Dense
import librosa
import pathlib
import numpy as np
import pandas as pd
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from librosa import display
import pywt
import numpy as np
import matplotlib.pyplot as plt


# Signal Transformation
def signal_transformation(path):
    y, sr = librosa.load(path, sr = 42666, duration = 3)
    file_length = np.size(y)
    if file_length < 128000:
        y = np.concatenate((y, np.zeros(128000-file_length)), axis=0)
    else:
        y=y[0:128000]
    
    mfcc = librosa.feature.mfcc(y = y, sr = sr, hop_length = 250, n_mfcc = 128)
    stft = np.abs(librosa.stft(y = y, n_fft = 254, hop_length = 250))
    chroma = librosa.feature.chroma_stft(S = stft, n_chroma = 128)

    #print(shape(mfcc))
    #print(shape(stft))
    #print(shape(chroma))
    
    log_mel_spectrogram = np.concatenate((mfcc,stft,chroma), axis=1)
    mfcc =[]
    stft =[]
    chroma=[]
    chro_cqt=[]
    cqt =[]
    y=[]
    
    log_mel_spectrogram = log_mel_spectrogram.reshape((-1,))
    return log_mel_spectrogram

def classify_files(path):
    dataset_dict = {
        'total': 0,
        'file_dict': {
            '1-1': {'represent': 0, 'count': 0, 'all_data': []},
            '2-1': {'represent': 1, 'count': 0, 'all_data': []},
            '2-2': {'represent': 2, 'count': 0, 'all_data': []},
            '3-1': {'represent': 3, 'count': 0, 'all_data': []},
            '3-2': {'represent': 4, 'count': 0, 'all_data': []},
            '4-1': {'represent': 5, 'count': 0, 'all_data': []},
            '4-2': {'represent': 6, 'count': 0, 'all_data': []},
            '5-1': {'represent': 7, 'count': 0, 'all_data': []},
            '5-2': {'represent': 8, 'count': 0, 'all_data': []}
        }
    } 
        
    wav_path = pathlib.Path(path)
    emotion_file_list = [str(file_name) for file_name in wav_path.glob('*.wav')]
    emotion_file_list.sort()
    #print(emotion_file_list)
    p = len(str(wav_path))

    emotion_label_list = dataset_dict['file_dict'].keys()
    for emotion_label in emotion_label_list:
        print(emotion_label)
        # emotion_classify_file_list = [letter for letter in emotion_file_list if letter[p+1:p+4] == emotion_label and letter[p+5:p+6] =='0']
        emotion_classify_file_list = [letter for letter in emotion_file_list if letter[p+1:p+4] == emotion_label]
        ck = 0
        new_list = []
        #female: 0, 49
        indf = 0
        #male: 50, 99
        indm = 50
        for i in range(0, 10):
            for j in range(0, 10):
                if (ck == 0):
                    #print(indf)
                    new_list.append(emotion_classify_file_list[indf])
                    indf = indf + 1
                else:
                    #print(indm)
                    new_list.append(emotion_classify_file_list[indm])
                    indm = indm + 1
            ck ^= 1

        emotion_classify_file_list = new_list
        print(emotion_classify_file_list)
        files_count = len(emotion_classify_file_list)

        dataset_dict['file_dict'][emotion_label]['count'] = files_count
        dataset_dict['total'] = dataset_dict['total'] + files_count
        emotion_data = [signal_transformation(path) for path in emotion_classify_file_list] 
        print(shape(emotion_data))
        dataset_dict['file_dict'][emotion_label]['all_data'] = emotion_data 
        log_mel_spectrogram =[]
        emotion_data = []

    return dataset_dict

def load_data(path):
    train_data_x = []
    train_data_y = []
    test_data_x = []
    test_data_y = []

    dataset_dict = classify_files(path)

    emotion_label_list = dataset_dict['file_dict'].keys()
    fold=5
    temp=0
    for emotion_label in emotion_label_list:
        x = dataset_dict['file_dict'][emotion_label]['all_data']
        count = dataset_dict['file_dict'][emotion_label]['count']
        y = np.full(count, dataset_dict['file_dict'][emotion_label]['represent'])
        
        x = np.array(x)
        y = np.array(y)
        kfold = KFold(n_splits = 5, random_state = None, shuffle = False)
        foldCount = 1
        for train, test in kfold .split(x, y):
            if (foldCount == fold):
                print(train)
                print(test)
                print(fold)
                train_data_x = np.append(train_data_x, x[train])
                train_data_y = np.append(train_data_y, y[train])
                test_data_x = np.append(test_data_x, x[test])
                test_data_y = np.append(test_data_y, y[test])
                x_train=[] 
                x_test=[]
                y_train=[]
                y_test =[]
                x=[]
                y=[]
                break
            else:
                foldCount = foldCount + 1
     
   
    dataset_dict=[]
   
    np.savetxt(path +'/data/train_data_y.csv', train_data_y, delimiter=',')
    np.savetxt(path +'/data/test_data_y.csv', test_data_y, delimiter=',')
    
    x = 128
    y = 513
    z = 3
    train_data_x = np.array(train_data_x).reshape(-1, x, y,z,1)
    train_data_y = np.array(train_data_y)
    test_data_x = np.array(test_data_x).reshape(-1, x, y,z,1)
    test_data_y = np.array(test_data_y)

    return train_data_x,train_data_y,test_data_x,test_data_y 


path='/content/drive/MyDrive/Thesis/Dataset/Bangla/All'

train_data_x,train_data_y,test_data_x,test_data_y =load_data(path)

1-1
['/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-001.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-002.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-003.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-004.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-005.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-006.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-007.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-008.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-009.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-0-010.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-1-001.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-1-002.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-1-003.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-1-004.wav', '/content/drive/MyDrive/Thesis/Dataset/Bangla/All/1-1-1-005.wav', '/con

# Model Generate

In [ ]:
from numpy.core.fromnumeric import shape
import librosa
import pathlib
import numpy as np
import pywt
import csv
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# from google.colab import drive
import tensorflow as tf
from tensorflow.keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow import keras,nn
from tensorflow.keras import layers
import keras.backend as K
from sklearn.model_selection import cross_validate

###################################################################################################
def model3d(input_shape, num_classes):
    model = keras.Sequential(name='model3d')
    # FB1
    model.add(layers.Conv3D(filters=64, kernel_size=5, strides=1, padding='same', input_shape = input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1))) 
    # FB2
    model.add(layers.Conv3D(filters=64, kernel_size=5, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))
    # FB3
    model.add(layers.Conv3D(filters=128, kernel_size=5, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1))) 
    # FB4
    model.add(layers.Conv3D(filters=128, kernel_size=5, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling3D(pool_size=(4,4,1), strides=(4,4,1)))

    model.add(layers.TimeDistributed(layers.Flatten()))
    
    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(LSTM(256))

    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

############################################################################################################
def model2d(input_shape, num_classes):
    model = keras.Sequential(name='model2d')
    #LFLB1
    model.add(layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    #LFLB2
    model.add(layers.Conv2D(filters=64, kernel_size=5, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #LFLB3
    model.add(layers.Conv2D(filters=128, kernel_size=5, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    #LFLB4
    model.add(layers.Conv2D(filters=128, kernel_size=5, strides=1, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    model.add(layers.TimeDistributed(layers.Flatten()))

    model.add(Bidirectional(LSTM(256, return_sequences=True)))
    model.add(LSTM(256))

    model.add(Dense(num_classes, activation='softmax'))

    model.summary()
    opt = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])
    return model


def model7CNNFC(input_shape, num_classes):
    model = keras.Sequential(name='model7cnnfc')

    # FB1
    model.add(layers.Conv2D(filters=16,kernel_size=(7,7),strides=2,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    # FB2
    model.add(layers.Conv2D(filters=32,kernel_size=(5,5),strides=2,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    # FB3
    model.add(layers.Conv2D(filters=32,kernel_size=(3,3),strides=2,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    # FB4
    model.add(layers.Conv2D(filters=64,kernel_size=(3,3),strides=2,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    # FB5
    model.add(layers.Conv2D(filters=64,kernel_size=(3,3),strides=2,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    # FB6
    model.add(layers.Conv2D(filters=128,kernel_size=(3,3),strides=2,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    # FB7
    model.add(layers.Conv2D(filters=128,kernel_size=(3,3),strides=2,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation=nn.relu))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()

    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)

    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model


def model4CNNLSTM(input_shape, num_classes):
    model = keras.Sequential(name='model4cnnlstm')
    # FB1
    model.add(layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    # FB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    # FB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    # FB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('elu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    model.add(layers.Reshape((-1, 256)))

    #LSTM
    model.add(layers.LSTM(256))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()
    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model



def model4CNNTDFBLSTM(input_shape, num_classes):
    model = keras.Sequential(name='model2d')
    # FB1
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1,padding='same',input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=2, strides=2))
    # FB2
    model.add(layers.Conv2D(filters=64,kernel_size=3,strides=1, padding='same', ))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    # FB3
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))
    # FB4
    model.add(layers.Conv2D(filters=128,kernel_size=3,strides=1,padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D(pool_size=4, strides=4))

    model.add(layers.TimeDistributed(layers.Flatten()))
    model.add(layers.Bidirectional(layers.LSTM(512)))
    model.add(layers.Dense(units=num_classes, activation='softmax'))

    model.summary()
    opt = keras.optimizers.Adam(learning_rate=0.0006, decay=1e-6)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['categorical_accuracy'])

    return model


def train(train_data_x, train_data_y,test_data_x, test_data_y,emotion,emotionNumber):
    model = model3d(input_shape=(128, 513, 3, 1), num_classes=emotionNumber)
    # model = model2d(input_shape=(128, 513, 3), num_classes=emotionNumber)
    # model = model7CNNFC(input_shape=(128, 513, 1), num_classes=emotionNumber)
    # model = model4CNNLSTM(input_shape=(128, 513, 1), num_classes=emotionNumber)
    # model = model4CNNTDFBLSTM(input_shape=(128, 513, 1), num_classes=emotionNumber)
    
    es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=0, patience=20)
    mc = ModelCheckpoint(path+'/model/'+emotion+'_max_model.h5',monitor='val_categorical_accuracy',mode='max',verbose=0,save_best_only=True)
    history=model.fit(train_data_x, train_data_y, validation_data=(test_data_x, test_data_y), epochs=100, batch_size=16, verbose=0, callbacks=[es,mc])

    # los = history.history['loss']
    # vlos = history.history['val_loss']
    # lvl = 'Loss_vs_Epochs_Proposed Model_' + emotion
    # datas = {'Train_Loss':los, 'Test_Loss':vlos}
    # lsvls = pd.DataFrame(datas)
    # lsvls.to_csv(f'/content/drive/MyDrive/Thesis/CSV/{lvl}.csv', index = False)

    # acc = history.history['categorical_accuracy']
    # vacc = history.history['val_categorical_accuracy']
    # ava = 'Accuracy_vs_Epochs_Proposed Model_' + emotion
    # datas = {'Train_Accuracy':acc, 'Test_Accuracy':vacc}
    # lsvls = pd.DataFrame(datas)
    # lsvls.to_csv(f'/content/drive/MyDrive/Thesis/CSV/{ava}.csv', index = False)


    model=[]
    acc=history.history['categorical_accuracy']
    vacc=history.history['val_categorical_accuracy']
    loss=history.history['loss']
    vloss=history.history['val_loss']
    accLoss=[]
    accLoss.append(acc)
    accLoss.append(vacc)
    accLoss.append(loss)
    accLoss.append(vloss)
    acc=history.history['categorical_accuracy'][len(history.history['categorical_accuracy']) - 1]
    accLoss=[]
    model=[]
    # model.save(path+'/model/'+emotion+'_model.h5')
    return acc

def test(test_data_x, test_data_y,emotion):
    new_model = load_model(path+'/model/'+emotion+'_max_model.h5')
    history=new_model.evaluate(test_data_x, test_data_y, batch_size=1)
    predict=new_model.predict(test_data_x)
    return history[1]

def maxIndex(data):
    max=data[0]
    index=0
    for i in range(1,len(data)):
        if(max<data[i]):
            max=data[i]
            index=i
    return index  

def test_emotion(test_data_x, test_data_y,total,emotion):
    new_model = load_model(path+'/model/'+emotion+'_max_model.h5')
    predict=new_model.predict(test_data_x)
    test_data_y=np.argmax(test_data_y, axis=1)
    temp=total+2
    count=[[0]*temp]*temp
    count=np.array(count)
    #print(count)
    for i in range(0,len(test_data_y)):
        predictNew=maxIndex(predict[i])
        #print(str(test_data_y[i])+' -- '+ str(predictNew))
        count[test_data_y[i]][predictNew]=count[test_data_y[i]][predictNew]+1
    for i in range(0,total):
        for j in range(0,total):
            if(i==j):
                count[i][total]=count[i][j]
                count[i][total+1]=sum(count[i][0:total])-count[i][j]

    with open(path +'/data/'+emotion+'_confution.csv',"w+") as my_csv:
        csvWriter = csv.writer(my_csv,delimiter=',')
        csvWriter.writerows(count)
    return count

#5 Emotion

In [ ]:
def lebelTo5(data):
    print(len(data))
    for i in range(len(data)):
        if(int(data[i])%2!=0):
            data[i]=int(data[i])+1
        data[i]=data[i]/2
    return data
train_data_y = np.array(np.genfromtxt(path +'/data/train_data_y.csv', delimiter=','))
#validation_data_y = np.array(np.genfromtxt(path +'/data/validation_data_y.csv', delimiter=','))
test_data_y = np.array(np.genfromtxt(path +'/data/test_data_y.csv', delimiter=','))
train_data_y = to_categorical(lebelTo5(train_data_y))
#validation_data_y = to_categorical(lebelTo5(validation_data_y))
test_data_y = to_categorical(lebelTo5(test_data_y))

emotion='5_emotion'
#crossValidationTrain(train_data_x, train_data_y,5)

acc=train(train_data_x, train_data_y,test_data_x, test_data_y,emotion,5)
result = test_emotion(test_data_x, test_data_y,5,emotion)
result = np.array(result)

print("-----------------------------------------------------------------")
accuracy=[]; 
for i in range(0,5):
    accuracy.append([0]*8);
total_data=0
total_correct=0
for i in range(0,5):
    for j in range(0,8):
        if(j!=7):
            accuracy[i][j]=result[i][j]
        else:
            total=result[i][j-1]+result[i][j-2]
            total_data+=total
            total_correct+=result[i][j-2]
            accuracy[i][j]=round(result[i][j-2]*100/total, 2)
            
for i in range(0,len(accuracy)):
    print(accuracy[i])

print("Accuracy: ", end = "")
print(round(total_correct*100/total_data, 2))

720
180
Model: "model3d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 128, 513, 3, 64)   8064      
                                                                 
 batch_normalization (BatchN  (None, 128, 513, 3, 64)  256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 128, 513, 3, 64)   0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 64, 256, 3, 64)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 64, 256, 3, 64)    512064    
                                                                 
 batch_normalization_1 (Batc  (None, 64, 256, 3, 64

#4 Emotion 2 Grade

In [ ]:
# 2 emotion or intensity
def train_2(train_data_x, train_data_y,test_data_x, test_data_y,emotion,emotionNumber):
    model = model3d(input_shape=(128, 513, 3, 1), num_classes=emotionNumber)
    # model = model2d(input_shape=(128, 513, 3), num_classes=emotionNumber)
    # model = model7CNNFC(input_shape=(128, 513, 1), num_classes=emotionNumber)
    # model = model4CNNLSTM(input_shape=(128, 513, 1), num_classes=emotionNumber)
    # model = model4CNNTDFBLSTM(input_shape=(128, 513, 1), num_classes=emotionNumber)
    
    es = EarlyStopping(monitor='val_categorical_accuracy',mode='max',verbose=0,patience=20)
    mc = ModelCheckpoint(path+'/model/'+emotion+'_max_model.h5',monitor='val_categorical_accuracy',mode='max',verbose=0,save_best_only=True)
    history=model.fit(train_data_x, train_data_y, validation_data=(test_data_x, test_data_y), epochs=100, batch_size=16,verbose=0, callbacks=[es,mc])
    
    acc=history.history['categorical_accuracy']
    vacc=history.history['val_categorical_accuracy']
    loss=history.history['loss']
    vloss=history.history['val_loss']
    accLoss=[]
    accLoss.append(acc)
    accLoss.append(vacc)
    accLoss.append(loss)
    accLoss.append(vloss)

    acc=history.history['categorical_accuracy'][len(history.history['categorical_accuracy']) - 1]
    accLoss=[]
    model=[]
    #model.save(path+'/model/'+emotion+'_model.h5')
    return acc


def predictTest(test_data_x, test_data_y_5,test_data_y_3,sl):
    new_model = load_model(path+'/model/5_emotion_max_model.h5')
    predict=new_model.predict((test_data_x))
    
    tempTestX=[]
    tempTestY=[]
    count=0
    for i in range(len(test_data_y_5)):
        predict_5=maxIndex(predict[i])
        if(predict_5==sl and predict_5==test_data_y_5[i]):
            tempTestX.append(test_data_x[i])
            tempTestY.append(test_data_y_3[i])
            count=count+1
    tempTestX=np.array(tempTestX)
    tempTestY=np.array(tempTestY)

    return tempTestX, tempTestY


def labelTo2All(train_data_y,train_data_x,label):
    label=label*2
    tempX=[]
    tempY=[]
    for i in range(len(train_data_y)):
        if(int(train_data_y[i])+1==label):
            tempY.append(0)
            tempX.append(train_data_x[i])
        elif(int(train_data_y[i])== label):
            tempY.append(1)
            tempX.append(train_data_x[i])

    tempX=np.array(tempX)
    tempY=np.array(tempY)
    return tempX,tempY


def lebelTo5(data):
    print(len(data))
    for i in range(len(data)):
        if(int(data[i])%2!=0):
            data[i]=int(data[i])+1
        data[i]=data[i]/2
    return data


def lebelTo3(train_data_y):
    tempX=[]
    tempY=[]
    for i in range(len(train_data_y)):
        if(int(train_data_y[i])%2==0 and int(train_data_y[i])!=0):
            train_data_y[i]=1
        elif(int(train_data_y[i])!=0):
            train_data_y[i]=0
        else:
            train_data_y[i]=2
    return train_data_y


emotion = ['neutral','happy','sad','angry','disgust']
all = []
for i in range(1,len(emotion)):
    train_data_y = np.array(np.genfromtxt(path +'/data/train_data_y.csv', delimiter=','))
    tempTrainX,tempTrainY = labelTo2All(train_data_y,train_data_x,i)

    test_data_y = np.array(np.genfromtxt(path +'/data/test_data_y.csv', delimiter=','))
    test_data_y_5 = lebelTo5(test_data_y)

    test_data_y = np.array(np.genfromtxt(path +'/data/test_data_y.csv', delimiter=','))
    test_data_y_3 = lebelTo3(test_data_y)
    
    tempTestX, tempTestY = predictTest(test_data_x, test_data_y_5,test_data_y_3,i)
    acc = train_2(tempTrainX, to_categorical(tempTrainY),tempTestX,to_categorical(tempTestY),emotion[i],2)
    res = test_emotion(tempTestX, to_categorical(tempTestY),2,emotion[i])
    res = np.array(res)
    all.append(res)

id = 0
for result in all:
    id += 1
    if id == 1:
        print("Happy")
    elif id == 2:
        print("Sad")
    elif id == 3:
        print("Angry")
    elif id == 4:
        print("Disgust")
    
    accuracy=[]; 
    for i in range(0,2):
        accuracy.append([0]*5);
    total_data=0
    total_correct=0
    for i in range(0,2):
        for j in range(0,5):
            if(j!=4):
                accuracy[i][j]=result[i][j]
            else:
                total=result[i][j-1]+result[i][j-2]
                total_data+=total
                total_correct+=result[i][j-2]
                accuracy[i][j]=round(result[i][j-2]*100/total, 2)

    for i in range(0,len(accuracy)):
        print(accuracy[i])
        
    print("Accuracy: ", end = "")
    print(round(total_correct*100/total_data, 2))
    print("----------------------------------------------------------------")

180
6/6 [==============================] - 4s 410ms/step
Model: "model3d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_24 (Conv3D)          (None, 128, 513, 3, 64)   8064      
                                                                 
 batch_normalization_24 (Bat  (None, 128, 513, 3, 64)  256       
 chNormalization)                                                
                                                                 
 activation_24 (Activation)  (None, 128, 513, 3, 64)   0         
                                                                 
 max_pooling3d_24 (MaxPoolin  (None, 64, 256, 3, 64)   0         
 g3D)                                                            
                                                                 
 conv3d_25 (Conv3D)          (None, 64, 256, 3, 64)    512064    
                                                                 
 b

#9 Emotion

In [ ]:
train_data_y = np.array(np.genfromtxt(path +'/data/train_data_y.csv', delimiter=','))
#validation_data_y = np.array(np.genfromtxt(path +'/data/validation_data_y.csv', delimiter=','))
test_data_y = np.array(np.genfromtxt(path +'/data/test_data_y.csv', delimiter=','))

train_data_y = to_categorical(train_data_y)
#validation_data_y = to_categorical(validation_data_y)
test_data_y = to_categorical(test_data_y)
emotion='9_emotion'
#acc,vacc=train(train_data_x, train_data_y, validation_data_x, validation_data_y,emotion,9)
acc=train(train_data_x, train_data_y,test_data_x, test_data_y,emotion,9)

emotion = '9_emotion'
result = test_emotion(test_data_x, test_data_y,9,emotion)
result = np.array(result)

print("-----------------------------------------------------------------")
accuracy=[]; 
for i in range(0,9):
    accuracy.append([0]*12);
total_data=0
total_correct=0
for i in range(0,9):
    for j in range(0,12):
        if(j!=11):
            accuracy[i][j]=result[i][j]
        else:
            total=result[i][j-1]+result[i][j-2]
            total_data+=total
            total_correct+=result[i][j-2]
            accuracy[i][j]=round(result[i][j-2]*100/total, 2)
            
for i in range(0,len(accuracy)):
    print(accuracy[i])

print("Accuracy: ", end = "")
print(round(total_correct*100/total_data, 2))

Model: "model3d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 128, 513, 3, 64)   8064      
                                                                 
 batch_normalization_4 (Batc  (None, 128, 513, 3, 64)  256       
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 128, 513, 3, 64)   0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 64, 256, 3, 64)   0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 64, 256, 3, 64)    512064    
                                                                 
 batch_normalization_5 (Batc  (None, 64, 256, 3, 64)   256 